In [ ]:
!%pip install tensorflow
!%pip install scikit-learn
!%pip install matplotlib
!%pip install pandas
!%pip install pydot

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib as imp
import datacleaner
import datetime
imp.reload(datacleaner)

from datacleaner import *

In [57]:
np.random.seed(1)
tf.keras.backend.clear_session()

bd = pd.read_csv(r"training_data.csv", na_filter=False)

In [58]:
timesteps = 7 #number of days that make up a sequence
univariate = 1 #number of features used by the model (using incidents to predict incidents)
multisteps = 7 #number of days to forecast – we will forecast the next 7 days
cv_splits = 3 #time series cross validator
epochs = 50
batch_size = 7 #7 sequences of 7 days - which corresponds to a window of 7 days in a batch

In [59]:
bd.head()

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain,incidents
0,Guimaraes,UNDEFINED,0,",",2021-03-15 23:00,DARK,12.0,1013.0,70.0,1.0,0.0,Sem Chuva,None
1,Guimaraes,UNDEFINED,385,"N101,",2021-12-25 18:00,DARK,12.0,1007.0,91.0,1.0,0.0,Sem Chuva,None
2,Guimaraes,UNDEFINED,69,",",2021-03-12 15:00,LIGHT,14.0,1025.0,64.0,0.0,0.0,Sem Chuva,Low
3,Guimaraes,MAJOR,2297,"N101,R206,N105,N101,N101,N101,N101,N101,N101,N...",2021-09-29 09:00,LIGHT,15.0,1028.0,75.0,1.0,0.0,Sem Chuva,Very_High
4,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",2021-06-13 11:00,LIGHT,27.0,1020.0,52.0,1.0,0.0,Sem Chuva,High


In [60]:
def prepare_data(bd):
    bd = ordernar(bd)
    # bd = RoadsCleaner(bd)
    #bd = data(bd)
    # bd = valores_em_falta(bd)
    # bd = eliminar(bd)
    bd = incidentsNumbers(bd)
    # bd = luminosidade(bd)
    # bd = rainNumbers(bd)
    # bd = delayNumbers(bd)
    bd = removeOutlier(bd)
    bd.index = pd.to_datetime(bd["record_date"], format="%Y-%m-%d %H:%M:00")
    bd = bd[['incidents']]
    return bd

def data_normalization(bd, norm_range=(-1, 1)):
    scaler = MinMaxScaler(feature_range=norm_range)
    bd[['incidents']] = scaler.fit_transform(bd[['incidents']].values)
    return scaler

def to_supervised(df, timesteps):
    data = df.values
    X, y = list(), list()
    dataset_size = len(data)
    for curr_pos in range(dataset_size):
        input_index = curr_pos + timesteps
        label_index = input_index + 1

        if label_index < dataset_size:
            X.append(data[curr_pos:input_index,:])
            y.append(data[input_index:label_index, 0])
    
    return np.array(X).astype('float32'), np.array(y).astype('float32')

def split_data(bd, perc=10):
    train_idx = np.arange(0, int(len(bd)*(100-perc)/100))
    val_idx = np.arange(int(len(bd)*(100-perc)/100+1), len(bd))
    return train_idx, val_idx

In [61]:
bd = prepare_data(bd)

/home/diogofrosario/Desktop/trabalho-CSC/datacleaner.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd["delay_in_seconds"][count] = valor
/home/diogofrosario/Desktop/trabalho-CSC/datacleaner.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd["delay_in_seconds"][count] = valor
/home/diogofrosario/Desktop/trabalho-CSC/datacleaner.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd["delay_in_seconds"][count] = valor
/home/d

In [62]:
bd.head()

,incidents
record_date,
2021-01-16 16:00:00,3
2021-01-16 18:00:00,2
2021-01-16 19:00:00,2
2021-01-16 20:00:00,2
2021-01-16 21:00:00,3


In [63]:
np.unique(bd.incidents)

array([0, 1, 2, 3, 4])

In [64]:
scaler = data_normalization(bd)
bd.head()

,incidents
record_date,
2021-01-16 16:00:00,0.5
2021-01-16 18:00:00,0.0
2021-01-16 19:00:00,0.0
2021-01-16 20:00:00,0.0
2021-01-16 21:00:00,0.5


In [65]:
print(bd)

                     incidents
record_date                   
2021-01-16 16:00:00        0.5
2021-01-16 18:00:00        0.0
2021-01-16 19:00:00        0.0
2021-01-16 20:00:00        0.0
2021-01-16 21:00:00        0.5
...                        ...
2021-12-31 17:00:00        0.0
2021-12-31 18:00:00       -1.0
2021-12-31 19:00:00       -1.0
2021-12-31 20:00:00       -1.0
2021-12-31 23:00:00       -1.0

[5000 rows x 1 columns]


In [69]:
bd['incidents'] = bd['incidents'].resample("H").interpolate("time")
bd = bd[bd.index.month !=8]
print(bd)

                     incidents
record_date                   
2021-01-16 16:00:00        0.5
2021-01-16 18:00:00        0.0
2021-01-16 19:00:00        0.0
2021-01-16 20:00:00        0.0
2021-01-16 21:00:00        0.5
...                        ...
2021-12-31 17:00:00        0.0
2021-12-31 18:00:00       -1.0
2021-12-31 19:00:00       -1.0
2021-12-31 20:00:00       -1.0
2021-12-31 23:00:00       -1.0

[5000 rows x 1 columns]


In [ ]:
fig=plt.figure()
plt.plot(bd)
spacing =10
fig.subplots_adjust(right=spacing)
plt.show()

In [ ]:
X, y = to_supervised(bd, timesteps)
X.shape, y.shape

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
def rmse(y_true, y_pred):
    return tf.keras.backend.sqrt(tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true)))

def build_model(timesteps, features, filters=16, kernel_size=5, pool_size=2):
    #using the Functional API
    inputs = tf.keras.layers.Input(shape=(timesteps, features))
    print(inputs)
    #microarchitecture
    x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size,
    activation='relu', data_format='channels_last')(inputs)
    x = tf.keras.layers.AveragePooling1D(pool_size=pool_size,
    data_format='channels_first')(x)
    #last layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(filters)(x)
    outputs = tf.keras.layers.Dense(1)(x)
    #the model
    cnnModel = tf.keras.Model(inputs=inputs, outputs=outputs, name='cnn_model')
    tf.keras.utils.plot_model(cnnModel, 'cnn.png', show_shapes=True)
    return cnnModel

def plot_learning_curves(history, epochs):
    #accuracies and losses
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(epochs)
    #creating figure
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training/Validation Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training/Validation Loss')
    plt.show()

def plot_train_data(data, approach='history'):
    plt.figure(figsize=(8,6))
    if approach == 'history':
        plt.title('Model train vs val loss per Training Split')
        plt.ylabel('Training RMSE (Normalized)')
        plt.xlabel('Epoch')
        for hist, i in zip(data, range(len(data))):  
            plt.subplot(3,1,i+1)      
            plt.plot(hist.epoch, hist.history['loss'])
            plt.plot(hist.epoch, hist.history['val_loss'])
            plt.xlim([0, max(hist.epoch)])  
            plt.legend(['Training split ' + str(i+1) + ' - train loss', 'Training split ' + str(i+1) + ' - val loss'], loc='upper right')
        plt.show()   
    elif approach == 'loss':
        plt.figure(figsize=(6,3))
        plt.plot(range(len(data)), data)
        plt.title('RMSE value per Timeseries split')
        plt.ylabel('Evaluation RMSE')
        plt.xlabel('Timeseries Splits')
        plt.xlim([0, 2])
        plt.ylim([0, (np.amax(data)+2)])
        plt.show()

def compile_and_fit(model, epochs, batch_size):
    #compile
    model.compile(loss = rmse, optimizer = tf.keras.optimizers.Adam(), metrics = ['mae', rmse])
    #fit
    hist_list = list()
    loss_list = list()
    #Time Series Cross Validator
    tscv = TimeSeriesSplit(n_splits=cv_splits)
    for train_index, test_index in tscv.split(X):
        train_idx, val_idx = split_data(train_index, perc=10) #further split into training and validation sets
        #build data
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        X_test, y_test = X[test_index], y[test_index]
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
        epochs=epochs, batch_size=batch_size, shuffle=False)
        metrics = model.evaluate(X_test, y_test)
        hist_list.append(history)
        loss_list.append(metrics[2])
    plot_train_data(hist_list, approach='history')
    # plot_learning_curves(loss_list, approach='loss')
    return model, hist_list, loss_list

def forecast(model, df, timesteps, multisteps, scaler):
    input_seq = df[-timesteps:].values #getting the last sequence of known value
    inp = input_seq
    predictions = list()
    for _ in range(1, multisteps+1):
        inp = inp.reshape(1, timesteps, 1)
        yhat = model.predict(inp)
        yhat_inversed = scaler.inverse_transform(yhat)
        predictions.append(yhat_inversed[0][0])
        #prepare new input to forecast the next day
        inp = np.append(inp[0], yhat)
        inp = inp[-timesteps:]
    return predictions

def plot_forecast(data, forecasts):
    plt.figure(figsize=(8,6))
    plt.plot(range(len(data)), data, color='green', label='Confirmed')
    plt.plot(range(len(data)-1, len(data)+len(forecasts)-1), forecasts, color='red', label='Forecasts')
    plt.title('Number of incidents')
    plt.ylabel('Incidents')
    plt.xlabel('Days')
    plt.legend()
    plt.show()    

In [ ]:
model = build_model(timesteps, univariate)

In [ ]:
model, hist_list, loss_list = compile_and_fit(model, epochs, batch_size)

In [ ]:
forecasts = forecast(model, bd, timesteps, multisteps, scaler)
plot_forecast(bd, forecasts)

In [ ]:
forecasts

In [ ]:
model.compile(loss=rmse, optimizer=tf.keras.optimizers.Adam(), metrics=['mae', rmse])
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle=False)

In [ ]:

# plot_learning_curves(hist_list, epochs)

In [ ]:
forecasts = forecast(model, bd, timesteps, multisteps, scaler)

In [ ]:
print(forecasts)

In [ ]:
plot_forecast(bd, forecasts)